# Financial News Classification

This Notebook integrates with Mubasher.info to perform sentiment analysis on the latest news titles. 
The analysis classifies sentiments into three categories:

1- Positive
2- Neutral
3- Negative

## Web scraping
To get the titles from latest news on Mubasher.info

**Note:** This web scraping code is suitable for Mubasher.info if you want to change the website you should change the letter ('a' and class_ ) in soup.find_all('a', class_='mi-article-media-block__title') 
as appropriate to the new website.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import Asset_Management as am 

In [9]:
import requests
from bs4 import BeautifulSoup

url = 'https://english.mubasher.info/news/eg/now/latest'

try:
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find elements containing news article titles
    title_elements = soup.find_all('a', class_='mi-article-media-block__title') #if you change the url change 'a' and class_ as appropriate to the new website.

    # Extract and store the titles in a list
    news_titles = [title.text.strip() for title in title_elements if title.text.strip()]

    # Print the list of news titles

except requests.exceptions.RequestException as e:
    print('Failed to retrieve the web page:', e)
except Exception as e:
    print('An error occurred:', e)
for title in news_titles:
        print(title)

EBE’s consolidated profits jump to EGP 3.4bn in 2023
Egypt Kuwait Holding logs $218m consolidated net profit in 2023
TMG Holding partners with ADQ, Modon Properties to develop $24bn Ras El-Hekma project
Banque du Caire anchors MSMEs in Egypt via $50m financing
CBE auctions T-bills worth EGP 62bn on Sunday
Abu Dhabi-based ADQ to invest $35bn in Egypt’s Ras El-Hekma
EGX indices end higher on Thursday; market cap approaches EGP 3trn
Kima’s profits fall 15% in H1-23/24
Emirates NBD-Egypt inks EGP 400m with ADIB’s leasing arm
CBE issues EGP 70bn T-bills on Thursday
SCMF 2024 marks launch of Single Stock Options contracts on SNB
Saudi Arabia could form majority of region’s MSCI EM – Goldman Sachs at Saudi Capital Market Forum
nybl, Basserah merge to boost AI industry worldwide
Egyptian Exchange maintains varied performance through Wednesday
DAL Developments to open 1st project in Egypt early March
Beltone Holding signs agreement to facilitate expansion in Africa
National Investment Bank lowe

## Sentiment analysis 
we will import DistilRoberta-financial-sentiment model from Hugging face. it is an AI model to make sentiment analysis for the financial news.
you can get more information about the model form here: https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

In [4]:
model_id = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForSequenceClassification.from_pretrained(model_id)

In [5]:
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
sentiment_result = sentiment_task(news_titles)


### Put the Titles and Sentiment Result in one DataFrame 

In [6]:
data = pd.DataFrame(sentiment_result)
data.index = range(1, len(data) + 1)
data.insert(0, 'News Titles', news_titles)
data

,News Titles,label,score
1,EBE’s consolidated profits jump to EGP 3.4bn i...,positive,0.999731
2,Egypt Kuwait Holding logs $218m consolidated n...,neutral,0.999655
3,"TMG Holding partners with ADQ, Modon Propertie...",neutral,0.999856
4,Banque du Caire anchors MSMEs in Egypt via $50...,neutral,0.999887
5,CBE auctions T-bills worth EGP 62bn on Sunday,neutral,0.999891
6,Abu Dhabi-based ADQ to invest $35bn in Egypt’s...,neutral,0.999828
7,EGX indices end higher on Thursday; market cap...,positive,0.994586
8,Kima’s profits fall 15% in H1-23/24,negative,0.998627
9,Emirates NBD-Egypt inks EGP 400m with ADIB’s l...,neutral,0.999885
10,CBE issues EGP 70bn T-bills on Thursday,neutral,0.999892


### Statistics Of Results

In [7]:
stat = am.sentiments_stat(data)
stat

,Counts,Percentages
Positive,8,32.0
Neutral,15,60.0
Negative,2,8.0
Total,25,100.0
